In [1]:
import pandas as pd
from pulp import *
import numpy as np
import os

## Dataset of the city of Mallorca

In [5]:
df = pd.read_csv("mallorca.csv",index_col ="year")
df.head(10)
n = df.shape[0] 
p = df.shape[1]

## Optimization problem

In [6]:
def conterfactuals_global_indicator(epsilon, max_incr, mu_u, mu_l, m, q, X_year, weight_v, df, year_selected,k):

  beta = []
  names_beta = ["beta" + str(j) for j in range(1, p+1)]
  for j in range(p):
      betaj = LpVariable(names_beta[j]) 
      beta.append(betaj)

  delta = []
  names_delta = ["delta" + str(j) for j in range(1, p+1)]
  for j in range(p):
      deltaj = LpVariable(names_delta[j], cat='Binary') # binary variables
      delta.append(deltaj)

  model = LpProblem("SustainableTourism", LpMinimize)

  for j in range(p):
      model += (beta[j] <= max_incr[j])
      model += (m * delta[j] <= beta[j])
      model += (beta[j] <= q * delta[j])
      model += (beta[j] + X_year[j] <= 1)

  model += (sum(delta) <= mu_u)
  model += (mu_l <= sum(delta))

  for j in range(0,p):
      aux = [ (X_year[j] + beta[j]) * weight_v[j] - ((X_year[j]) * weight_v[j]) for j in range(0,p)]
      auxi = sum(aux)
  model += (auxi >= epsilon)

  # objective function
  obj_func = sum(beta)
  model += obj_func + k*sum(delta)

  # Return results and feasability
  status = model.solve()
  print("status:", LpStatus[status])
  print("values:")
  beta_values = np.array([value(beta[i]) for i in range(p)])
  #print("\tbeta vector:",beta_values)
  delta_values = np.array([value(delta[i]) for i in range(p)])
  #print("\tdelta vector:",delta_values)

  res = pd.DataFrame({'beta': beta_values, 'delta': delta_values}, index=df.columns[:-1])


  var_update = df.iloc[year_selected,:-1]+beta_values

  # y'
  y_prima = var_update.dot(weight_v)
  y_prima

  increase = y_prima - df.iloc[year_selected,-1]

  return res, increase


## Definition of parameters established by the expert in the tourism domain

In [7]:
# Standard deviation of each variable
std_vector = df.std(axis=0)
std_vector
# Weights
weight_v = [1/p] * p
# Global indicator
df['y'] = df.dot(weight_v)

## Experiment 1

In [8]:
year_selected = 24 # Year 2024
data_year = df.iloc[year_selected,:] # with global indicator y
X_year = df.iloc[year_selected,:-1] # withouth global indicator y
data_year

x1    0.590476
x2    0.484559
x3    0.520416
x4    0.148170
x5    0.829295
x6    0.756693
x7    0.800650
x8    0.331331
y     0.557699
Name: 2024, dtype: float64

In [9]:
# Parameters and constants
epsilon = 0.01*data_year['y']
max_incr = 0.15*std_vector
mu_u = 1
mu_l = 1
m = -50000
q = 50000
k=0

In [10]:
res, increase = conterfactuals_global_indicator(epsilon, max_incr, mu_u, mu_l, m, q, X_year, weight_v, df, year_selected,k)

status: Optimal
values:


In [11]:
# Changes
print(res)

        beta  delta
x1  0.000000    0.0
x2  0.000000    0.0
x3  0.000000    0.0
x4  0.000000    0.0
x5  0.000000    0.0
x6  0.000000    0.0
x7  0.000000    0.0
x8  0.044616    1.0


In [12]:
# Increase in y
print(increase)

0.00557698824999997


## Experiment 2

In [13]:
year_selected = 23 # Year 2023
data_year = df.iloc[year_selected,:] # with global indicator y
X_year = df.iloc[year_selected,:-1] # withouth global indicator y
data_year

x1    0.714286
x2    0.527860
x3    0.428610
x4    0.007442
x5    0.741528
x6    1.000000
x7    1.000000
x8    0.485874
y     0.613200
Name: 2023, dtype: float64

In [14]:
# Parameters and constants
epsilon = 0.05*data_year['y']
max_incr = 0.3*std_vector
mu_u = 4
mu_l = 1
m = -50000
q = 50000
k = 10

In [15]:
res, increase = conterfactuals_global_indicator(epsilon, max_incr, mu_u, mu_l, m, q, X_year, weight_v, df, year_selected,k)

status: Optimal
values:


In [16]:
# Changes
print(res)

        beta  delta
x1  0.091358    1.0
x2  0.048782    1.0
x3  0.000000    0.0
x4  0.000000    0.0
x5  0.000000    0.0
x6  0.000000    0.0
x7  0.000000    0.0
x8  0.105140    1.0


In [17]:
# Increase in y
print(increase)

0.030660003749999998


## Experiment 3

In [18]:
year_selected = 23 # Year 2023
data_year = df.iloc[year_selected,:] # with global indicator y
X_year = df.iloc[year_selected,:-1] # withouth global indicator y
data_year

x1    0.714286
x2    0.527860
x3    0.428610
x4    0.007442
x5    0.741528
x6    1.000000
x7    1.000000
x8    0.485874
y     0.613200
Name: 2023, dtype: float64

In [19]:
# Parameters and constants
epsilon = 0.25*data_year['y']
max_incr = 0.9*std_vector
mu_u = 6
mu_l = 1
m = -50000
q = 50000
k = 10

In [20]:
res, increase = conterfactuals_global_indicator(epsilon, max_incr, mu_u, mu_l, m, q, X_year, weight_v, df, year_selected,k)

status: Optimal
values:


In [21]:
# Changes
print(res)

        beta  delta
x1  0.140588    1.0
x2  0.271385    1.0
x3  0.241337    1.0
x4  0.000000    0.0
x5  0.257670    1.0
x6  0.000000    0.0
x7  0.000000    0.0
x8  0.315420    1.0


In [22]:
# Increase in y
print(increase)

0.15330001999999998
